# 0. Import packages

First, we import the necessary packages for this exercise.

In [2]:
import xarray as xr 
import matplotlib.pyplot as plt 
import cartopy.crs as ccrs
from datetime import datetime, timedelta
import numpy as np
import cartopy.feature as cfeature

# 1. Open simulation data

We have simulated this case for this exercise many times in many different configurations. All the output data can be found in the 'data' folder. The data is stored in netCDF-files, which can be easily read as xarray Datasets. Each of the netCDF-files has a structured name `<configuration>_<starttime>_<nhours>_<variable>_<tstart>_<tstop>_<tstep>_regridded.nc` where the different elements mean:
1. `<configuration>`: This refers to the set-up of the model. For this exercise, we will only use the data from the 'baseline'-simulation.
2. `<starttime>`: This refers to the start time of the simulation. By varying the start time, we can create an ensemble of different simulations of the same event (see below). The start time is written in the format of `<year><month><day><hour>`.
3. `<nhours>`: This refers to the number of hours that the simulation has run.
4. `<variable>`: This refers to the variable saved in this file. We use the standard abbreviations from the Climate and Forecast (CF) Metadata Conventions. The ones you will need in these exercises are:
    - *pr*: Precipitation
    - *tas*: Near-Surface Air Temperature
    - *tasmax*: Maximum Near-Surface Air Temperature
    - *tasmin*: Minimum Near-Surface Air Temperature
5. `<tstart>`: This refers to the first time step of the data in the file. This is written in the format of `<year>-<month>-<day>T<hour>`.
6. `<tstart>`: This refers to the last time step of the data in the file. This is written in the format of `<year>-<month>-<day>T<hour>`.
7. `<tstep>`: This refers to the time between subsequent time steps in the data, expressed in seconds. You will see that this is 3600 for all files (so hourly data).
8. `regridded`: This refers to the fact that the data has been regridded from the orginal coordinate reference system (CRS) to a latitude-longitude reference system. This makes the data easier to work with.
9. `.nc`: This extension denotes the fact that this is a netCDF-file.

Let's start by opening a file and loading in the data! Load in the precipitation data corresponding to the baseline simulation which started at midnight of 1 July 2021. You can use the xarray function `open_dataset` ([documentation](https://docs.xarray.dev/en/stable/generated/xarray.open_dataset.html)) **open_mfdataset??** Save this dataset as `ds` and inspect the contents by printing the output or simply running a cell which ends with `ds`. The following questions/tasks will help you along:
- What are the dimensions of the data?
- First consider the spatial dimensions. What are the units? What is the resolution? Convert this to (kilo)metres. Use the fact that the radius of the Earth is 6371 km.
- Next, consider the attributes of the variable `pr`. What are the units? Usually, precipitation flux is expressed in mm/h. Convert these units (if necessary) to mm/h. Use the fact that water has a density of 1000 $kg/m^3$ .
- Lastly, in the attributes of the variable `pr`, you can find `cell_methods = time: mean`. This signifies that the variable is not instantaneous value at a certain moment in time, but provides to average value over a certain time interval. The bounds of this interval are given in the coordinate `time_bnds`. So, every value of the coordinate `time` has two corresponding values in the coordinate `time_bnds`: the start and end time of the interval over which the average is taken. Compare the values in `time_bnds` to those in `time`. For a `time` value at moment $x$, which interval corresponds to this? What is the advantage of providing the average precipitation flux, instead of the instantaneous value?

In [3]:
## Write your code here

## Solutions:
- Dimensions are (time, latitude, longitude).
- The units are degrees. The resolution is 0.07° for longitude and 0.05° for latitude. We can convert this to kilometres at a certain latitude $\phi$, with $R$ the radius of the Earth:
    - $\Delta x = \left( \frac{\Delta \text{lon}}{360\degree} \right)$ * (circumference of circle of latitude) = $\left(\frac{\Delta \text{lon}}{360\degree}\right) (2 \pi R \cos \phi)$
    - $\Delta y = \left( \frac{\Delta \text{lat}}{360\degree} \right)$ * (circumference of circle of longitude) = $\left(\frac{\Delta \text{lat}}{360\degree}\right) (2 \pi R)$

    We can see that while $\Delta x$ depends on the latitude, $\Delta y$ does not. Filling in the numbers and using an average latitude of $50.5\degree$ we find: $\Delta x = 4.95$ km and $\Delta y = 5.56$ km
- We convert $\text{kg} / (\text{m}^2 \cdot \text{s})$ to $\text{mm}/\text{h}$:
    $$1 \frac{\text{kg}}{\text{m}^2 \cdot \text{s}} = \frac{10^{-3} \text{m}^3}{\text{m}^2 \cdot \frac{1}{3600} \text{h}} = 3600 \frac{10^{-3} \text{m}^3}{\text{m}^2 \text{h}} = 3600 \frac{10^{-3} \text{m}}{ \text{h}} = 3600 \frac{\text{mm}}{ \text{h}}$$
    So a precipitation flux expressed in $\text{kg} / (\text{m}^2 \cdot \text{s})$ needs to be multiplied by 3600 to convert it to $\text{mm}/\text{h}$.
- The value at time $x$ corresponds to the average of the interval $[x - 1 h, x]$. Providing the hourly average precipitation flux enables us to exactly calculate the total amount of precipitation, which would not be possible if only the hourly instantaneous flux was provided.

In [4]:
ds = xr.open_mfdataset("../data/baseline_20210701*pr*", engine="netcdf4")
ds

<xarray.Dataset>
Dimensions:    (time: 384, bnds: 2, lon: 75, lat: 70)
Coordinates:
  * time       (time) datetime64[ns] 2021-07-01T01:00:00 ... 2021-07-17
    time_bnds  (time, bnds) datetime64[ns] dask.array<chunksize=(1, 2), meta=np.ndarray>
  * lon        (lon) float64 2.0 2.07 2.14 2.21 2.28 ... 6.9 6.97 7.04 7.11 7.18
  * lat        (lat) float64 49.0 49.05 49.09 49.13 ... 51.97 52.02 52.06 52.1
Dimensions without coordinates: bnds
Data variables:
    pr         (time, lat, lon) float32 dask.array<chunksize=(1, 70, 75), meta=np.ndarray>
Attributes: (12/33)
    CDI:                       Climate Data Interface version 2.0.6 (https://...
    source:                    ALARO-1 (CY43T2) - SURFEX v8.0
    Conventions:               CF-1.11
    activity_id:               DD
    comment:                   
    contact:                   wout.dewettinck@ugent.be
    ...                        ...
    version_realization_info:  
    creation_date:             2024-10-25T11:53:17Z
    frequency:                 1hr
    StartTime:                 2021-07-01T00:00
    EndTime:                   2021-07-17T00:00
    CDO:                       Climate Data Operators version 2.0.6 (https://...

In [5]:
dlon = 0.07
dlat = 0.05
R = 6371
lat = 50.5

dx = (dlon / 360) * (2 * np.pi * R * np.cos(lat * np.pi / 180))
dy = (dlat / 360) * (2 * np.pi * R)

print(f"dx = {dx:.2f} km ; dx = {dy:.2f} km")

dx = 4.95 km ; dx = 5.56 km


In [6]:
def convert_to_mm_h(ds):
    if "pr" in ds:
        ds.pr.values = ds.pr.values * 3600
        ds.pr.attrs["units"] = "mm/h"

    return ds

ds = convert_to_mm_h(ds)
ds

<xarray.Dataset>
Dimensions:    (time: 384, bnds: 2, lon: 75, lat: 70)
Coordinates:
  * time       (time) datetime64[ns] 2021-07-01T01:00:00 ... 2021-07-17
    time_bnds  (time, bnds) datetime64[ns] dask.array<chunksize=(1, 2), meta=np.ndarray>
  * lon        (lon) float64 2.0 2.07 2.14 2.21 2.28 ... 6.9 6.97 7.04 7.11 7.18
  * lat        (lat) float64 49.0 49.05 49.09 49.13 ... 51.97 52.02 52.06 52.1
Dimensions without coordinates: bnds
Data variables:
    pr         (time, lat, lon) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
Attributes: (12/33)
    CDI:                       Climate Data Interface version 2.0.6 (https://...
    source:                    ALARO-1 (CY43T2) - SURFEX v8.0
    Conventions:               CF-1.11
    activity_id:               DD
    comment:                   
    contact:                   wout.dewettinck@ugent.be
    ...                        ...
    version_realization_info:  
    creation_date:             2024-10-25T11:53:17Z
    frequency:                 1hr
    StartTime:                 2021-07-01T00:00
    EndTime:                   2021-07-17T00:00
    CDO:                       Climate Data Operators version 2.0.6 (https://...

In [7]:
time = ds.time[0]
time_bnds = ds.time_bnds[0]

print(f"time : {time.values}")
print(f"time bounds : {time_bnds.values}")

time : 2021-07-01T01:00:00.000000000
time bounds : ['2021-07-01T01:00:00.000000000' '2021-07-01T01:00:00.000000000']


# 2. Plot the accumulated precipitation (simulation)

- Plot the accumulated precipitation between 8h on 13/07/2021 and 8h on 15/07/2021.
- Use the skeleton code from below!
- Aspect ratio?

## Solutions: